In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import requests
import pandas as pd
from src.tables import get_machines
from src.utils import df_na_vals

In [43]:
raw = get_machines()

In [44]:
cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = raw.cpu_name.str.replace('Core™', '')
raw.cpu_name = cpu_name
raw['cores'] = (raw.cpu_cores / raw.num_gpus).fillna(0).astype(int)
raw['ram'] = (raw.cpu_ram / raw.num_gpus / 1024).round().astype(int)
raw['dph'] = raw.dph_base / raw.num_gpus
raw['disk'] = (raw.disk_space / raw.num_gpus).astype(int)
raw['rented_fraq'] = '[' + raw.num_gpus_rented.astype(str) + "/" + raw.num_gpus.astype(str) + ']'
raw['country'] = raw.location.dropna().apply(lambda loc: loc['country'])
# raw['rented'] = raw.num_gpus_rented > 0
raw['rented'] = raw.num_gpus_rented == raw.num_gpus
raw['min_fraq'] = raw.min_chunk/raw.num_gpus

raw.dlperf = raw.dlperf.round(1)
raw.dlperf_per_dphtotal = raw.dlperf_per_dphtotal.round(1)
raw.disk_bw = raw.disk_bw.round()
raw.duration = pd.to_timedelta(raw.duration * 1e9).dt.days
raw.inet_down = raw.inet_down.fillna(0).astype(int)
raw.inet_up = raw.inet_up.fillna(0).astype(int)
raw.inet_up_cost = (raw.inet_up_cost * 1000).fillna(0).astype(int)
raw.inet_down_cost = (raw.inet_down_cost * 1000).fillna(0).astype(int)

raw['isp'] = raw.location.apply(lambda x: x.get('isp', None) if not pd.isna(x) else None)

# raw.end_date = (pd.to_datetime(raw.end_date * 1e9) - pd.Timestamp.now()).dt.days
raw.end_date = pd.to_datetime(raw.end_date * 1e9)
raw.start_date = pd.to_datetime(raw.start_date * 1e9)
# raw.set_index('machine_id', inplace=True)

C:\Users\ergot\AppData\Local\Temp\ipykernel_24148\2497115936.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_24148\2497115936.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'^AMD','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_24148\2497115936.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'Processor$','')


In [45]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'dlperf_per_dphtotal', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability2', 'duration', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'country', 'isp']

raw.query('dph >= 0.5')[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country,isp
45,H100 PCIE,[0/2],4.2000,0.2,24.4,96,252,6379,4849.0,2,...,12.2,535.113.01,5,16,NaN,53.9,EPYC 9654 96C,GENOAD8UD,TW,Chunghwa Telecom
450,H100 PCIE,[4/4],3.5000,0.2,29.2,64,244,2533,17715.0,4,...,12.2,535.104.05,4,16,NaN,26.6,EPYC 7763 64C,ROME2D32GM,TW,Chunghwa Telecom
705,H100 PCIE,[1/2],3.0000,0.4,33.6,64,118,1078,3186.0,1,...,12.2,535.54.03,4,16,NaN,21.5,EPYC 7742 64C,MZ52,US,WebNX
464,RTX 3090,[0/4],2.0000,0.2,11.7,16,31,402,3239.0,1,...,12.2,535.104.05,3,8,NaN,12.4,Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,NO,Altibox AS
1111,RTX 3090,[0/6],1.8333,0.4,17.7,5,42,9,726.0,1,...,12.2,535.104.12,4,1,NaN,24.9,Ryzen TR PRO 3955WX 16Cs,M12SWA,CA,Bell Canada Business
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,RTX 4090,[2/2],0.5000,0.4,150.9,24,126,400,1913.0,1,...,12.2,535.86.05,4,16,NaN,22.8,EPYC 7402 24C,MZ32,CA,Bell Canada
988,A40,[4/5],0.5000,0.4,61.7,3,25,29,900.0,1,...,12.0,525.125.06,3,16,1.0,11.9,EPYC 7252 8C,Default string,SE,84 Grams AB
1424,RTX 4090,[1/1],0.5000,0.1,127.0,24,63,3176,6009.0,1,...,12.0,525.125.06,4,16,NaN,24.3,Ryzen 9 5900X 12C,B550M PRO,CA,Bell Canada
1437,RTX 4090,[2/2],0.5000,0.4,148.3,16,12,3138,2387.0,1,...,12.0,525.105.17,4,8,NaN,12.8,Ryzen 9 5950X 16C,PRIME X570,US,Verizon Fios


In [46]:
raw.query('num_gpus == 14')[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country,isp
620,RTX 4090,[12/14],0.40,0.1,170.8,13,72,425,4344.0,1,...,12.2,535.86.05,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,TW,Chunghwa Telecom
651,RTX 4090,[12/14],0.40,0.1,170.8,13,72,399,3166.0,1,...,12.2,535.104.05,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,TW,Chunghwa Telecom
658,RTX 4090,[11/14],0.40,0.1,180.5,13,36,419,5258.0,1,...,12.2,535.86.05,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,TW,Chunghwa Telecom
741,RTX 4090,[14/14],0.55,0.2,61.6,18,36,214,5102.0,14,...,12.2,535.113.01,4,16,NaN,12.6,EPYC 7742 64C,ROME2D32GM,TW,Chunghwa Telecom
825,RTX 4090,[0/14],0.65,0.4,66.6,18,36,200,1459.0,4,...,12.2,535.113.01,4,4,NaN,12.7,EPYC 7H12 64C,ROME2D32GM,US,Cox Communications
827,RTX 4090,[14/14],0.70,0.4,103.0,18,36,62,3111.0,1,...,12.2,535.113.01,4,4,NaN,12.7,EPYC 7662 64C,ROME2D32GM,US,Cox Communications
1052,RTX 4090,[14/14],0.50,0.2,61.3,13,36,467,6409.0,14,...,12.0,525.105.17,4,8,NaN,12.7,EPYC 7R12 48C,ROME2D32GM,TW,Chunghwa Telecom
1091,RTX 4090,[14/14],0.55,0.2,58.3,18,36,100,4832.0,14,...,12.2,535.113.01,4,8,NaN,24.0,EPYC 7742 64C,ROME2D32GM,TW,Chunghwa Telecom


In [51]:
raw[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country,isp
19,RTX 3090,[0/8],0.450,0.40,71.4,12,63,796,2887.0,1,...,12.2,535.113.01,4,16,NaN,21.3,EPYC 7402 24C,H12DSG,RU,ITSOFT LLC
94,RTX 3090,[0/1],0.307,0.20,112.8,128,252,806,5816.0,1,...,12.2,535.104.12,4,16,NaN,22.1,EPYC 7742 64C,ROMED8,RU,Moscow City Telephone Network
291,RTX 4090,[0/1],0.900,0.40,82.5,128,252,1591,3742.0,1,...,12.2,535.86.05,4,16,NaN,23.7,EPYC 7742 64C,H12SSL,RU,Rostelecom
314,RTX 3080 Ti,[3/3],0.140,0.05,177.3,10,42,1077,5599.0,1,...,12.2,535.104.05,4,4,NaN,6.1,Ryzen 9 5950X 16C,ROG STRIX X570,RU,Rostelecom
467,RTX 4090,[2/2],0.400,0.40,189.2,64,31,748,4115.0,1,...,12.2,535.104.05,4,16,NaN,24.4,EPYC 7B13 64C,ROMED8,RU,2com
478,RTX 3080 Ti,[1/2],0.140,0.05,178.3,18,63,1655,3344.0,1,...,12.2,535.113.01,3,16,NaN,11.8,Xeon® E5-2696 v3,X99,RU,Rostelecom
527,RTX 3080,[0/6],0.072,0.40,63.2,2,3,52,402.0,1,...,12.0,525.125.06,1,1,NaN,0.1,Core™ i5-10400F,H510 Pro BTC,RU,Iskratelecom CJSC
653,RTX 3090,[1/1],0.250,0.40,131.2,12,31,672,783.0,1,...,12.2,535.104.12,4,16,NaN,25.0,Ryzen 5 3600 6C,B550M DS3H,RU,Rostelecom
711,RTX 3090,[3/3],0.196,0.40,156.2,10,21,987,6148.0,1,...,12.0,525.125.06,4,4,NaN,5.9,Ryzen 9 5950X 16C,ROG STRIX X570,RU,Rostelecom
806,RTX 3090,[4/4],0.400,0.10,81.9,14,63,285,1206.0,1,...,12.0,525.105.17,3,16,NaN,10.6,Xeon® E5-2680 v4,Z10PG,RU,Delta Ltd
